In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch
import numpy as np
from sklearn.decomposition import DictionaryLearning
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'transformers'

In [ ]:
def interact_with_llm(prompt, model_name="decapoda-research/llama-7b-hf", max_length=100):
    # Load pre-trained model and tokenizer
    tokenizer = LlamaTokenizer.from_pretrained(model_name)
    model = LlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

    # Prepare input
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # Generate
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
        outputs_with_hidden = model(input_ids, output_hidden_states=True)

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Get the last hidden state
    last_hidden_state = outputs_with_hidden.hidden_states[-1]

    return generated_text, last_hidden_state


In [ ]:
def perform_dictionary_learning(data, n_components=15, alpha=1, max_iter=100, visualize=True):
    """
    Perform dictionary learning on the input data.
    
    Parameters:
    - data: numpy array, shape (n_samples, n_features)
    - n_components: int, number of dictionary elements to learn
    - alpha: float, sparsity controlling parameter
    - max_iter: int, maximum number of iterations
    - visualize: bool, whether to visualize the results
    
    Returns:
    - dictionary: learned dictionary
    - sparse_code: sparse representation of data
    """
    # Perform dictionary learning
    dl = DictionaryLearning(n_components=n_components, alpha=alpha, max_iter=max_iter, random_state=0)
    dictionary = dl.fit(data).components_
    
    # Transform data using the learned dictionary
    sparse_code = dl.transform(data)
    
    if visualize:
        visualize_results(data, dictionary, sparse_code)
    
    # Print some statistics
    print(f"Original data shape: {data.shape}")
    print(f"Learned dictionary shape: {dictionary.shape}")
    print(f"Sparse codes shape: {sparse_code.shape}")
    print(f"Average sparsity: {np.mean(sparse_code != 0):.2%}")
    
    return dictionary, sparse_code

In [ ]:
def visualize_results(data, dictionary, sparse_code):
    """
    Visualize the original data, learned dictionary, and sparse codes.
    """
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original data
    ax1.imshow(data.T, aspect='auto', interpolation='nearest')
    ax1.set_title("Original Data")
    ax1.set_xlabel("Samples")
    ax1.set_ylabel("Features")
    
    # Learned dictionary
    ax2.imshow(dictionary, aspect='auto', interpolation='nearest')
    ax2.set_title("Learned Dictionary")
    ax2.set_xlabel("Components")
    ax2.set_ylabel("Features")
    
    # Sparse codes
    ax3.imshow(sparse_code.T, aspect='auto', interpolation='nearest')
    ax3.set_title("Sparse Codes")
    ax3.set_xlabel("Samples")
    ax3.set_ylabel("Components")
    
    plt.tight_layout()
    plt.show()